In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import math
import pickle
import numpy as np
import pandas as pd

from collections import OrderedDict
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.cluster import KMeans

In [3]:
os.chdir('..')

In [4]:
from mofiwo.utility import (
    load_rna_fasta_zipfile,
    generate_utr_from_cdna_cds
)

In [5]:
def retrieve_seq(
    downloadloc:str,
    cdna_file:str,
    cds_file:str)->dict:
    
    # Load sequences from zip file
    seq_cdna = load_rna_fasta_zipfile(os.path.join(downloadloc, cdna_file))
    seq_cds = load_rna_fasta_zipfile(os.path.join(downloadloc, cds_file))

    # Classify CDS, UTR region
    seqs = dict()
    for k, v in generate_utr_from_cdna_cds(seq_cdna, seq_cds).items():
        seqs.update({k: {'CDS': seq_cds[k].seq, 'UT3': v['utr3'], 'UT5': v['utr5']}})
    
    return seqs

In [6]:
def get_feature_id(
    df_s1:pd.DataFrame,
    min_bval:float=0.5, 
    min_mean_obs:float=3.5, 
    max_qval:float=0.2,
    choice_prob:float=0.8, # Randomly selected
    used_id:list=None) -> list:
    
    if used_id is not None:
        df_s1 = df_s1[df_s1.target_id.apply(lambda x: x not in list(used_id))]
    
    feature_id = df_s1[(abs(df_s1.b) > min_bval) & 
                       (df_s1.mean_obs > min_mean_obs) & 
                       (df_s1.qval < max_qval)
                      ].target_id
    feature_id = np.random.choice(feature_id, int(len(feature_id) * choice_prob), replace=False)
    return feature_id

In [7]:
def generate_ml_data(seqs, df_s1, length_motif=100, window_size=50, nt_mapper ={'N':1000,'A':1001,'C':1002,'G':1003,'T':1004}):

    tgt_id = get_feature_id(df_s1)
    used_id = list()
    ml_data = OrderedDict()

    for key, val in seqs.items():
        _utr3 = seqs[key]['UT3']
        if len(_utr3.seq) > length_motif and key in tgt_id:
            used_id.append(key)
            _data = {i: str(_utr3.seq[i*window_size:(i*window_size)+length_motif]) for i in range(math.floor(len(_utr3)/window_size)-1)}

            seqs[key]['MTF'] = _data
            ml_data.update({f'{key}_{idx}': [nt_mapper[x] for x in nt] for idx, nt in _data.items()})
    
    return ml_data

In [8]:
def execute_kmeans(n_clusters=40):
    preprocessor = Pipeline(
        [
            ("scaler", MinMaxScaler()),
        ]
    )
    clusterer = Pipeline(
        [
            (
                "kmeans",
                KMeans(
                    n_clusters = n_clusters,
                    init = "k-means++",
                    n_init = 50,
                    max_iter = 500,
                    random_state = 42,
                ),
            ),
        ]
    )
    X = np.array([x for x in ml_data.values()])
    pipe = Pipeline(
        [
            ("preprocessor", preprocessor),
            ("clusterer", clusterer)
        ]
    )
    pipe.fit(X)
    
    return pipe

In [9]:
def generate_kmeans_report(ml_data,df_s1, pipe):
    _rpt = pd.DataFrame([list(ml_data.keys()), pipe['clusterer']['kmeans'].labels_]).T
    _rpt.columns = ['motif','cluster']
    _rpt['target'] = _rpt.motif.apply(lambda x: x[:x.find('_')])
    _rpt_tbl = _rpt.pivot_table(index='target', columns='cluster', values='motif', aggfunc= len)
    _rpt_tbl = pd.merge(_rpt_tbl / _rpt_tbl, df_s1['b'], how='inner', left_index=True, right_index=True)
    return _rpt_tbl

In [ ]:
downloadloc = os.path.join(os.path.expanduser('~'), r'workspace\rnamotif\samples')
seqs = retrieve_seq(downloadloc,'s1_cdna.zip','s1_cds.zip')
df_s1 = pd.read_excel(os.path.join(downloadloc,'aan2399_table_S1.xlsx'))
df_s1.index = df_s1.target_id

In [39]:
res = list()
for length_motif in [50,60,80,90]:
    for window_size in [30,40,50]:
        for n_clusters in [30,40,50,60]:
            try:
                ml_data = generate_ml_data(seqs, df_s1, length_motif=length_motif, window_size=window_size)
                pipe = execute_kmeans(n_clusters=n_clusters)
                rpt_tbl = generate_kmeans_report(ml_data,df_s1, pipe)
                rank_top10_apical = (rpt_tbl[rpt_tbl['b'] > 0][:-1].replace(np.nan, 0).apply(sum)[:-1] / len(rpt_tbl[rpt_tbl['b'] > 0])).sort_values(ascending=False)[:10]
                rank_top10_basal = (rpt_tbl[rpt_tbl['b'] < 0][:-1].replace(np.nan, 0).apply(sum)[:-1] / len(rpt_tbl[rpt_tbl['b'] < 0])).sort_values(ascending=False)[:10]
                #res.append({'length_motif': length_motif, 'window_size':window_size, 'n_clusters':n_clusters, 'top10_apical':rank_top10_apical, 'top10_basal':rank_top10_basal, 'rpt_tbl':rpt_tbl})
                df = pd.merge(pd.DataFrame([rank_top10_apical.index, rank_top10_apical.values]).T, pd.DataFrame([rank_top10_basal.index, rank_top10_basal.values]).T, left_index=True, right_index=True)
                df.columns = ['apical_number','apical_ratio','basal_number','basal_ratio']
                df['length_motif'] = length_motif
                df['window_size'] = window_size
                df['n_clusters'] = n_clusters
                res.append(df)
            except Exception as ex:
                print(f'error- length_motif({length_motif}), window_size({window_size}), n_clusters({n_clusters})')
                print(ex)
                continue

<ipython-input-8-fef6c4a6dccf>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([x for x in ml_data.values()])


error- length_motif(80), window_size(30), n_clusters(30)
setting an array element with a sequence.


<ipython-input-8-fef6c4a6dccf>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([x for x in ml_data.values()])


error- length_motif(80), window_size(30), n_clusters(40)
setting an array element with a sequence.


<ipython-input-8-fef6c4a6dccf>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([x for x in ml_data.values()])


...

error- length_motif(90), window_size(40), n_clusters(40)
setting an array element with a sequence.


<ipython-input-8-fef6c4a6dccf>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([x for x in ml_data.values()])


error- length_motif(90), window_size(40), n_clusters(50)
setting an array element with a sequence.


<ipython-input-8-fef6c4a6dccf>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array([x for x in ml_data.values()])


error- length_motif(90), window_size(40), n_clusters(60)
setting an array element with a sequence.


In [31]:
pickle.dump(res, open('../kmeans_result2.pickle', 'wb'))

In [41]:
pd.DataFrame([x.iloc[0,:] for x in res]).sort_values(['apical_ratio'], ascending=False)

,apical_number,apical_ratio,basal_number,basal_ratio,length_motif,window_size,n_clusters
0,17.0,0.704000,13.0,0.793651,60.0,30.0,30.0
0,24.0,0.692607,3.0,0.761905,50.0,30.0,30.0
0,30.0,0.636364,9.0,0.707692,50.0,30.0,40.0
0,3.0,0.634454,11.0,0.777778,80.0,40.0,30.0
0,20.0,0.627376,0.0,0.736842,50.0,40.0,30.0
0,27.0,0.621094,25.0,0.824561,60.0,30.0,40.0
0,25.0,0.615079,5.0,0.796610,60.0,40.0,30.0
0,15.0,0.605469,20.0,0.750000,60.0,50.0,30.0
0,1.0,0.592000,0.0,0.745455,80.0,50.0,30.0
0,19.0,0.580913,28.0,0.737705,50.0,50.0,30.0


In [25]:
_rpt = pd.DataFrame([list(ml_data.keys()), pipe['clusterer']['kmeans'].labels_]).T
_rpt.columns = ['motif','cluster']
_rpt['target'] = _rpt.motif.apply(lambda x: x[:x.find('_')])
#_rpt_tbl = _rpt.pivot_table(index='target', columns='cluster', values='motif', aggfunc= len)

In [27]:
_rpt_tbl = _rpt.pivot_table(index='target', columns='cluster', values='motif', aggfunc= len)

In [28]:
_rpt_tbl

cluster,0,1
target,,
ENSMUST00000000312,31.0,22.0
ENSMUST00000001319,49.0,55.0
ENSMUST00000001611,120.0,100.0
ENSMUST00000001963,9.0,21.0
ENSMUST00000002091,9.0,2.0
...,...,...
ENSMUST00000199397,32.0,34.0
ENSMUST00000199420,25.0,15.0
ENSMUST00000203652,6.0,13.0


In [16]:
res2 = res.copy()

In [29]:
res2[-1]

,apical_number,apical_ratio,basal_number,basal_ratio,length_motif,window_size,n_clusters
0,15.0,0.444882,45.0,0.631579,100,50,50
1,47.0,0.437008,26.0,0.561404,100,50,50
2,42.0,0.433071,21.0,0.561404,100,50,50
3,24.0,0.433071,35.0,0.561404,100,50,50
4,22.0,0.433071,15.0,0.561404,100,50,50
5,10.0,0.425197,42.0,0.561404,100,50,50
6,49.0,0.421260,36.0,0.543860,100,50,50
7,44.0,0.417323,37.0,0.543860,100,50,50
8,2.0,0.409449,40.0,0.543860,100,50,50
9,23.0,0.409449,0.0,0.543860,100,50,50


In [14]:
os.path.abspath(os.path.curdir)

'C:\\Users\\by003457\\workspace\\mofiwo'

In [140]:
import matplotlib.cm as cmap

In [ ]:
plt.rcParams['figure.figsize'] = [20, 40]
plt.imshow(bar3.to_numpy(), cmap=cmap.hot)